In [1]:
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt 
import torchvision 
import torchvision.transforms as transforms
import torch

In [2]:
train_dataset = torchvision.datasets.CIFAR10(root="./",train=True,download=True,transform=transforms.ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root="./",download=False,train=False,transform=transforms.ToTensor())

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./


In [3]:
batch_size = 128

In [4]:
train_dataset.data

array([[[[ 59,  62,  63],
         [ 43,  46,  45],
         [ 50,  48,  43],
         ...,
         [158, 132, 108],
         [152, 125, 102],
         [148, 124, 103]],

        [[ 16,  20,  20],
         [  0,   0,   0],
         [ 18,   8,   0],
         ...,
         [123,  88,  55],
         [119,  83,  50],
         [122,  87,  57]],

        [[ 25,  24,  21],
         [ 16,   7,   0],
         [ 49,  27,   8],
         ...,
         [118,  84,  50],
         [120,  84,  50],
         [109,  73,  42]],

        ...,

        [[208, 170,  96],
         [201, 153,  34],
         [198, 161,  26],
         ...,
         [160, 133,  70],
         [ 56,  31,   7],
         [ 53,  34,  20]],

        [[180, 139,  96],
         [173, 123,  42],
         [186, 144,  30],
         ...,
         [184, 148,  94],
         [ 97,  62,  34],
         [ 83,  53,  34]],

        [[177, 144, 116],
         [168, 129,  94],
         [179, 142,  87],
         ...,
         [216, 184, 140],
        

In [5]:
train_dataset.data.shape

(50000, 32, 32, 3)

In [6]:
train_dataset.targets

[6,
 9,
 9,
 4,
 1,
 1,
 2,
 7,
 8,
 3,
 4,
 7,
 7,
 2,
 9,
 9,
 9,
 3,
 2,
 6,
 4,
 3,
 6,
 6,
 2,
 6,
 3,
 5,
 4,
 0,
 0,
 9,
 1,
 3,
 4,
 0,
 3,
 7,
 3,
 3,
 5,
 2,
 2,
 7,
 1,
 1,
 1,
 2,
 2,
 0,
 9,
 5,
 7,
 9,
 2,
 2,
 5,
 2,
 4,
 3,
 1,
 1,
 8,
 2,
 1,
 1,
 4,
 9,
 7,
 8,
 5,
 9,
 6,
 7,
 3,
 1,
 9,
 0,
 3,
 1,
 3,
 5,
 4,
 5,
 7,
 7,
 4,
 7,
 9,
 4,
 2,
 3,
 8,
 0,
 1,
 6,
 1,
 1,
 4,
 1,
 8,
 3,
 9,
 6,
 6,
 1,
 8,
 5,
 2,
 9,
 9,
 8,
 1,
 7,
 7,
 0,
 0,
 6,
 9,
 1,
 2,
 2,
 9,
 2,
 6,
 6,
 1,
 9,
 5,
 0,
 4,
 7,
 6,
 7,
 1,
 8,
 1,
 1,
 2,
 8,
 1,
 3,
 3,
 6,
 2,
 4,
 9,
 9,
 5,
 4,
 3,
 6,
 7,
 4,
 6,
 8,
 5,
 5,
 4,
 3,
 1,
 8,
 4,
 7,
 6,
 0,
 9,
 5,
 1,
 3,
 8,
 2,
 7,
 5,
 3,
 4,
 1,
 5,
 7,
 0,
 4,
 7,
 5,
 5,
 1,
 0,
 9,
 6,
 9,
 0,
 8,
 7,
 8,
 8,
 2,
 5,
 2,
 3,
 5,
 0,
 6,
 1,
 9,
 3,
 6,
 9,
 1,
 3,
 9,
 6,
 6,
 7,
 1,
 0,
 9,
 5,
 8,
 5,
 2,
 9,
 0,
 8,
 8,
 0,
 6,
 9,
 1,
 1,
 6,
 3,
 7,
 6,
 6,
 0,
 6,
 6,
 1,
 7,
 1,
 5,
 8,
 3,
 6,
 6,
 8,
 6,
 8,
 4,
 6,
 6,


In [7]:
K = len(set(train_dataset.targets))

In [8]:
K

10

In [9]:
class CNN(nn.Module):
  def __init__(self,K):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(3,32,kernel_size=3,stride=2)
    self.conv2 = nn.Conv2d(32,64,kernel_size=3,stride=2)
    self.conv3 = nn.Conv2d(64,128,kernel_size=3,stride=2)
    
    self.l1 = nn.Linear(128*3*3,1024)
    self.l2 = nn.Linear(1024,K)
  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = F.relu(self.conv3(x))
    x = x.view(-1,128*3*3)
    x = F.dropout(x,p=0.5)
    x = F.relu(self.l1(x))
    x = F.dropout(x,p=0.2)
    x = self.l2(x)
    return x

In [10]:
model = CNN(K)
criterian = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [11]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [12]:
num_ephos = 120 
for ephos in range(num_ephos):
  for image,labels in train_loader:
    output = model(image)
    loss = criterian(output,labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f"ephos {ephos+1}/{num_ephos},loss {loss.item():.4f}")

Streaming output truncated to the last 5000 lines.
ephos 108/120,loss 0.3077
ephos 108/120,loss 0.3078
ephos 108/120,loss 0.2274
ephos 108/120,loss 0.2917
ephos 108/120,loss 0.2288
ephos 108/120,loss 0.1939
ephos 108/120,loss 0.3298
ephos 108/120,loss 0.1470
ephos 108/120,loss 0.3003
ephos 108/120,loss 0.2143
ephos 108/120,loss 0.3564
ephos 108/120,loss 0.3354
ephos 108/120,loss 0.3292
ephos 108/120,loss 0.3634
ephos 108/120,loss 0.3707
ephos 108/120,loss 0.2476
ephos 108/120,loss 0.2100
ephos 108/120,loss 0.1272
ephos 108/120,loss 0.2653
ephos 108/120,loss 0.2257
ephos 108/120,loss 0.3125
ephos 108/120,loss 0.2825
ephos 108/120,loss 0.3434
ephos 108/120,loss 0.3345
ephos 108/120,loss 0.2623
ephos 108/120,loss 0.2773
ephos 108/120,loss 0.2275
ephos 108/120,loss 0.2278
ephos 108/120,loss 0.3687
ephos 108/120,loss 0.2589
ephos 108/120,loss 0.3538
ephos 108/120,loss 0.2232
ephos 108/120,loss 0.2330
ephos 108/120,loss 0.3127
ephos 108/120,loss 0.3512
ephos 108/120,loss 0.3591
ephos 108/120

In [13]:
n_correct = 0
n_samples = 0
with torch.no_grad():
  for image,labels in train_loader:
    output = model(image)
    _,prediction = torch.max(output,1)
    n_correct += (prediction==labels).sum().item()
    n_samples += labels.shape[0]

acc = 100.0 * n_correct/n_samples

In [15]:
#Here saved the model for to use later also so do not need to run the training loop again
PATH = "state_dict_model.pt"

# Save
torch.save(model.state_dict(), PATH)

# Load
model2 = CNN(K)
model2.load_state_dict(torch.load(PATH))
model2.eval()

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (l1): Linear(in_features=1152, out_features=1024, bias=True)
  (l2): Linear(in_features=1024, out_features=10, bias=True)
)

In [16]:
n_correct = 0
n_samples = 0
with torch.no_grad():
  for image,labels in train_loader:
    output = model2(image)
    _,prediction = torch.max(output,1)
    n_correct += (prediction==labels).sum().item()
    n_samples += labels.shape[0]

acc1 = 100.0 * n_correct/n_samples

In [19]:
print(f"accuracy : {acc}")
print(f"accuracysaved : {acc1}")

accuracy : 91.48
accuracysaved : 91.462
